In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
from kaggle_secrets import UserSecretsClient
os.environ["GOOGLE_API_KEY"] = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_CSE_ID"] = UserSecretsClient().get_secret("GOOGLE_CSE_ID")
os.environ["OPENAI_API_KEY"] = UserSecretsClient().get_secret("OPENAI_API_KEY")

In [ ]:
!pip install -q langchain-openai langchain-community langchain-core requests duckduckgo-search

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
import requests

In [ ]:
%pip install --upgrade --quiet  langchain-google-community

In [ ]:
from langchain_core.tools import Tool
from langchain_google_community import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()

tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search.run,
)

In [ ]:
tool.run("Obama's first name?")

In [ ]:
import os
import requests
from langchain_core.tools import tool as lc_tool   # avoid name clash
from langchain_core.tools import Tool

@lc_tool
def get_weather_data(city: str) -> str:
    """Fetch current weather for a given city from Weatherstack"""
    access_key = os.getenv("WEATHERSTACK_API_KEY"," 2ba1a6e454385464848763c929c5efe2")  # or paste your key for testing
    url = f"http://api.weatherstack.com/current?access_key={access_key}&query={city}"
    r = requests.get(url, timeout=20)
    r.raise_for_status()
    data = r.json()
    if "error" in data:
        raise ValueError(data["error"].get("info", "Weather API error"))

    loc = data["location"]["name"]
    cur = data["current"]
    desc = cur["weather_descriptions"][0] if cur.get("weather_descriptions") else "N/A"
    return f"{loc}: {cur['temperature']}°C, {desc}"


In [ ]:
get_weather_data.run("Delhi")

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [ ]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [ ]:
# Step 2: Pull the ReAct prompt from LangChain Hub
prompt = hub.pull("hwchase17/react")  # pulls the standard ReAct agent prompt

In [ ]:
# Step 3: Create the ReAct agent manually with the pulled prompt
agent = create_react_agent(
    llm=llm,
    tools=[tool, get_weather_data],
    prompt=prompt
)

In [ ]:
# Step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[tool, get_weather_data],
    verbose=True
)

In [ ]:
# Step 5: Invoke
response = agent_executor.invoke({"input": "Find the capital of Madhya Pradesh, then find it's current weather condition"})
print(response)

In [ ]:
response['output']